### Query to get authors/commenters and counts where the subreddit = 'bigquery'

# How to select 10 subreddit's to analyze. 

select subreddit_id, count(unique(author)) as author_count
from [fh-bigquery:reddit_comments.2015_11]
group by subreddit_id
having author_count > 100 and author_count < 10000
order by 2 desc
limit 10





subreddit_id
(t5_2rmfx
t5_2ul7u
t5_2u3ta
t5_2r05i
t5_2qka0
t5_2yuej
t5_2qioo
t5_2qhqb
t5_2qh53
t5_2stl8

In [ ]:
# subreddits per author within the 10 subreddits. 

#Table : AUTHOR | COUNT

SELECT author, count(unique(subreddit)) as count_subs
FROM [fh-bigquery:reddit_comments.2015_05]
where subreddit_id in ('t5_2rmfx','t5_2ul7u','t5_2u3ta','t5_2r05i','t5_2qka0','t5_2yuej','t5_2qioo','t5_2qhqb','t5_2qh53','t5_2stl8')
group by author
having count_subs > 2

In [ ]:
# Returns AUTHOR | SUBREDDIT. 
SELECT author, subreddit
FROM [fh-bigquery:reddit_comments.2015_05]
where subreddit_id in ('t5_2rmfx','t5_2ul7u','t5_2u3ta','t5_2r05i','t5_2qka0','t5_2yuej','t5_2qioo','t5_2qhqb','t5_2qh53','t5_2stl8')
group by author, subreddit



# TO DO. 

1. JOIN TABLE OF AUTHOR | SUBREDDIT TO ITSELFT TO CREATE:
    
    AUTHOR1 | AUTHOR2 | SUBREDDIT AND 
    
2. RETURN JACAARD SIMILARITY. 

FOR EVERY PAIR OF AUTHORS... 
AUTHOR1 | AUTHOR2 | INTERSECTION | UNION | JACAARD.  

WHERE

@INTERSECTION = COUNT OF (AUTHOR1 | AUTHOR2 | SUBREDDIT )

@UNION = COUNT(AUTHOR1)+COUNT(AUTHOR2) - INTERSECTION. 

@JACAARD = INTERSECTION / UNION. 



In [ ]:
# Returns AUTHOR1 | Author2 | intersection | subreddits_author1 | subreddits_author1 | union | jaccard_similarity


# this is currently NOT working. PROBLEM Resources are exceed. 

select author1,author2, intersection, 
num_subreddit_author1,num_subreddit_author2,
(num_subreddit_author1+num_subreddit_author2)-intersection as union,
intersection/union as jaccard_similarity
from (
    select 
    author1,author2,count(distinct subreddit) as intersection,

    from (
        SELECT a.author as author1, b.author as author2, a.subreddit as subreddit
        FROM FLATTEN((
          SELECT UNIQUE(author) author, subreddit
          FROM [fh-bigquery:reddit_comments.2015_05]
          WHERE subreddit_id IN ('t5_2rmfx','t5_2ul7u','t5_2u3ta','t5_2r05i','t5_2qka0','t5_2yuej','t5_2qioo','t5_2qhqb','t5_2qh53','t5_2stl8')
           GROUP BY 2
        ),author) a
        left join 
        (FLATTEN((
          SELECT UNIQUE(author) author, subreddit
          FROM [fh-bigquery:reddit_comments.2015_05]
          WHERE subreddit_id IN ('t5_2rmfx','t5_2ul7u','t5_2u3ta','t5_2r05i','t5_2qka0','t5_2yuej','t5_2qioo','t5_2qhqb','t5_2qh53','t5_2stl8')
           GROUP BY 2
        ),author)) b on a.subreddit == b.subreddit 
        limit 10000000
     )
     group by 1,2
     having count(distinct subreddit) > 2
  limit 1000) a1a2
left join 
  (SELECT author, count(unique(subreddit)) as num_subreddit_author1
  FROM [fh-bigquery:reddit_comments.2015_05]
  WHERE subreddit_id IN ('t5_2rmfx','t5_2ul7u','t5_2u3ta','t5_2r05i','t5_2qka0','t5_2yuej','t5_2qioo','t5_2qhqb','t5_2qh53','t5_2stl8')
  group by author
  having num_subreddit_author1 >2) a1sub
on (a1a2.author1==a1sub.author)
left join 
  (SELECT author, count(unique(subreddit)) as num_subreddit_author2
  FROM [fh-bigquery:reddit_comments.2015_05]
  WHERE subreddit_id IN ('t5_2rmfx','t5_2ul7u','t5_2u3ta','t5_2r05i','t5_2qka0','t5_2yuej','t5_2qioo','t5_2qhqb','t5_2qh53','t5_2stl8')
  group by author
  having num_subreddit_author2 >2) a2sub
on (a1a2.author2==a2sub.author)
limit 5


In [ ]:
I need to calculate the total number of subreddits that either authors belongs too. 

The table looks like: Author1 | author2 | subreddit 

... 

the correct output should be : author1 | author2 | count_intersection_of_subreddits| count_union_of_subreddits 